In [2]:
import pandas as pd
df=pd.read_csv("AugustCancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
import numpy as np

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
rf_classifier = RandomForestClassifier(random_state=42)

In [6]:
X = df.drop(['id', 'diagnosis'], axis=1)  
y = df['diagnosis']

In [7]:
rf_classifier.fit(X,y)

RandomForestClassifier(random_state=42)

In [8]:
imp_features = rf_classifier.feature_importances_

In [9]:
sorted_feature_indices = np.argsort(imp_features)[::-1]

In [10]:
top_8_feature_indices = sorted_feature_indices[:8]

In [11]:
top_8_feature_names = X.columns[top_8_feature_indices]

In [12]:
print("Top 8 important features:", top_8_feature_names)

Top 8 important features: Index(['area_worst', 'concave points_worst', 'concave points_mean',
       'radius_worst', 'perimeter_worst', 'perimeter_mean', 'concavity_mean',
       'area_mean'],
      dtype='object')


In [13]:
X_important = df.iloc[:, top_8_feature_indices]

In [14]:
Y_important = df[['diagnosis']]

In [15]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le_diagnosis=LabelEncoder()

In [22]:
df['diagnosis_n'] = le_diagnosis.fit_transform(df['diagnosis'])

In [23]:
X_important_full = pd.concat([X_important, df['diagnosis_n']], axis=1)

In [24]:
X_important_full.head()

,texture_worst,compactness_worst,compactness_mean,symmetry_se,radius_worst,radius_mean,smoothness_mean,texture_mean,diagnosis_n
0,17.33,0.6656,0.27760,0.03003,25.38,17.99,0.11840,10.38,1
1,23.41,0.1866,0.07864,0.01389,24.99,20.57,0.08474,17.77,1
2,25.53,0.4245,0.15990,0.02250,23.57,19.69,0.10960,21.25,1
3,26.50,0.8663,0.28390,0.05963,14.91,11.42,0.14250,20.38,1
4,16.67,0.2050,0.13280,0.01756,22.54,20.29,0.10030,14.34,1


In [25]:
target_feature = X_important_full['diagnosis_n']

In [26]:
final_features = X_important_full.drop(['diagnosis_n'], axis=1)

In [27]:
final_features.head()

,texture_worst,compactness_worst,compactness_mean,symmetry_se,radius_worst,radius_mean,smoothness_mean,texture_mean
0,17.33,0.6656,0.27760,0.03003,25.38,17.99,0.11840,10.38
1,23.41,0.1866,0.07864,0.01389,24.99,20.57,0.08474,17.77
2,25.53,0.4245,0.15990,0.02250,23.57,19.69,0.10960,21.25
3,26.50,0.8663,0.28390,0.05963,14.91,11.42,0.14250,20.38
4,16.67,0.2050,0.13280,0.01756,22.54,20.29,0.10030,14.34


In [28]:
target_feature

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis_n, Length: 569, dtype: int32

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(final_features,target_feature,test_size=0.2)

In [30]:
len(X_train)

455

In [31]:
len(X_test)

114

In [35]:
len(final_features)


569

In [33]:
len(target_feature)

569

In [41]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()

In [42]:
model.fit(X_train,y_train)

GaussianNB()

In [43]:
model.score(X_test,y_test)

0.9649122807017544

In [44]:
model.predict([[11.0,0.5,0.2,0.04,28.0,18.0,0.14,11.0]])

/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([1])